[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain-academy/blob/main/module-4/research-assistant.ipynb) [![Open in LangChain Academy](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66e9eba12c7b7688aa3dbb5e_LCA-badge-green.svg)](https://academy.langchain.com/courses/take/intro-to-langgraph/lessons/58239974-lesson-4-research-assistant)

# Research Assistant

## Review

We've covered a few major LangGraph themes:

* Memory
* Human-in-the-loop
* Controllability

Now, we'll bring these ideas together to tackle one of AI's most popular applications: research automation. 

Research is often laborious work offloaded to analysts. AI has considerable potential to assist with this.

However, research demands customization: raw LLM outputs are often poorly suited for real-world decision-making workflows. 

Customized, AI-based [research and report generation](https://jxnl.co/writing/2024/06/05/predictions-for-the-future-of-rag/#reports-over-rag) workflows are a promising way to address this.

## Goal

Our goal is to build a lightweight, multi-agent system around chat models that customizes the research process.

`Source Selection` 
* Users can choose any set of input sources for their research.
  
`Planning` 
* Users provide a topic, and the system generates a team of AI analysts, each focusing on one sub-topic.
* `Human-in-the-loop` will be used to refine these sub-topics before research begins.
  
`LLM Utilization`
* Each analyst will conduct in-depth interviews with an expert AI using the selected sources.
* The interview will be a multi-turn conversation to extract detailed insights as shown in the [STORM](https://arxiv.org/abs/2402.14207) paper.
* These interviews will be captured in a using `sub-graphs` with their internal state. 
   
`Research Process`
* Experts will gather information to answer analyst questions in `parallel`.
* And all interviews will be conducted simultaneously through `map-reduce`.

`Output Format` 
* The gathered insights from each interview will be synthesized into a final report.
* We'll use customizable prompts for the report, allowing for a flexible output format. 

![Screenshot 2024-08-26 at 7.26.33 PM.png](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66dbb164d61c93d48e604091_research-assistant1.png)

## Setup

In [1]:
from langchain_ollama import ChatOllama
llm = ChatOllama(model="qwen3:8b", temperature=0)

We'll use [LangSmith](https://docs.langchain.com/langsmith/home) for [tracing](https://docs.langchain.com/langsmith/observability-concepts).

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

### Cell 1: Data Models and State Definitions
This cell defines the structure of your analysts and the shared memory (State) for the different graphs.

In [3]:
import operator
from typing import List, Annotated, Union
from typing_extensions import TypedDict
from pydantic import BaseModel, Field
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage, get_buffer_string

# --- Analyst Models ---
class Analyst(BaseModel):
    affiliation: str = Field(description="Primary affiliation of the analyst.")
    name: str = Field(description="Name of the analyst.")
    role: str = Field(description="Role of the analyst in the context of the topic.")
    description: str = Field(description="Description of the analyst focus, concerns, and motives.")
    
    @property
    def persona(self) -> str:
        return f"Name: {self.name}\nRole: {self.role}\nAffiliation: {self.affiliation}\nDescription: {self.description}\n"

class Perspectives(BaseModel):
    analysts: List[Analyst] = Field(description="Comprehensive list of analysts with their roles and affiliations.")

class SearchQuery(BaseModel):
    search_query: str = Field(None, description="Search query for retrieval.")

# --- Graph States ---
class GenerateAnalystsState(TypedDict):
    topic: str 
    max_analysts: int 
    human_analyst_feedback: str 
    analysts: List[Analyst] 

class InterviewState(TypedDict):
    messages: Annotated[List[Union[AIMessage, HumanMessage, SystemMessage]], operator.add]
    max_num_turns: int 
    context: Annotated[list, operator.add] 
    analyst: Analyst 
    interview: str 
    sections: list 

class ResearchGraphState(TypedDict):
    topic: str 
    max_analysts: int 
    human_analyst_feedback: str 
    analysts: List[Analyst] 
    sections: Annotated[list, operator.add] 
    introduction: str 
    content: str 
    conclusion: str 
    final_report: str

### Cell 2: Personas Generation Logic
This cell handles creating the AI analysts and managing human feedback.

In [4]:
from langgraph.graph import START, END, StateGraph
from langgraph.checkpoint.memory import MemorySaver

analyst_instructions = """You are tasked with creating a set of AI analyst personas. Follow these instructions carefully:
1. Review the research topic: {topic}
2. Examine editorial feedback: {human_analyst_feedback}
3. Determine interesting themes and pick the top {max_analysts} themes.
4. Assign one analyst to each theme."""

def create_analysts(state: GenerateAnalystsState):
    topic = state['topic']
    max_analysts = state['max_analysts']
    human_analyst_feedback = state.get('human_analyst_feedback', '')
        
    structured_llm = llm.with_structured_output(Perspectives)
    system_message = analyst_instructions.format(topic=topic,
                                                 human_analyst_feedback=human_analyst_feedback, 
                                                 max_analysts=max_analysts)

    res = structured_llm.invoke([SystemMessage(content=system_message)] + 
                                [HumanMessage(content="Generate the set of analysts.")])
    return {"analysts": res.analysts}

def human_feedback(state: GenerateAnalystsState):
    """ No-op node for interruption """
    pass

### Cell 3: The Interview Sub-Graph Nodes
This cell contains the specialized nodes for searching the web, Wikipedia, and conducting the dialogue.

In [5]:
from langchain_tavily import TavilySearch
from langchain_community.document_loaders import WikipediaLoader

tavily_search = TavilySearch(max_results=3)

def search_web(state: InterviewState):
    structured_llm = llm.with_structured_output(SearchQuery)
    res = structured_llm.invoke([search_instructions] + state['messages'])
    query = getattr(res, 'search_query', None)
    
    if not query: return {"context": []}

    data = tavily_search.invoke({"query": query})
    search_docs = data.get("results", data) if isinstance(data, dict) else data
    
    formatted = "\n\n---\n\n".join([f'<Document href="{d.get("url","")}"/>\n{d.get("content","")}\n</Document>' for d in search_docs])
    return {"context": [formatted]} 

def search_wikipedia(state: InterviewState):
    structured_llm = llm.with_structured_output(SearchQuery)
    res = structured_llm.invoke([search_instructions] + state['messages'])
    query = getattr(res, 'search_query', None)

    if not query: return {"context": []}

    try:
        docs = WikipediaLoader(query=query, load_max_docs=2).load()
        formatted = "\n\n---\n\n".join([f'<Document source="{d.metadata.get("source","")}" page="{d.metadata.get("page", "")}"/>\n{d.page_content}\n</Document>' for d in docs])
        return {"context": [formatted]}
    except:
        return {"context": []}

def generate_question(state: InterviewState):
    analyst = state["analyst"]
    system_message = question_instructions.format(goals=analyst.persona)
    question = llm.invoke([SystemMessage(content=system_message)] + state["messages"])
    return {"messages": [question]}

def generate_answer(state: InterviewState):
    analyst = state["analyst"]
    system_message = answer_instructions.format(goals=analyst.persona, context=state["context"])
    answer = llm.invoke([SystemMessage(content=system_message)] + state["messages"])
    answer.name = "expert"
    return {"messages": [answer]}

def save_interview(state: InterviewState):
    return {"interview": get_buffer_string(state["messages"])}

def route_messages(state: InterviewState, name: str = "expert"):
    messages = state["messages"]
    num_responses = len([m for m in messages if isinstance(m, AIMessage) and getattr(m, 'name', '') == name])
    if num_responses >= state.get('max_num_turns', 2) or "Thank you so much for your help" in messages[-2].content:
        return 'save_interview'
    return "ask_question"

/home/abdul/langchain/3-langgraph_intro/.venv/lib/python3.12/site-packages/langchain_tavily/tavily_research.py:97: UserWarning: Field name "output_schema" in "TavilyResearch" shadows an attribute in parent "BaseTool"
  class TavilyResearch(BaseTool):  # type: ignore[override, override]
/home/abdul/langchain/3-langgraph_intro/.venv/lib/python3.12/site-packages/langchain_tavily/tavily_research.py:97: UserWarning: Field name "stream" in "TavilyResearch" shadows an attribute in parent "BaseTool"
  class TavilyResearch(BaseTool):  # type: ignore[override, override]


### Cell 4: Writing and Finalizing the Report
This cell handles the "Reduce" phase, where individual analyst memos are turned into a coherent document.

In [6]:
def write_section(state: InterviewState):
    system_message = section_writer_instructions.format(focus=state["analyst"].description)
    section = llm.invoke([SystemMessage(content=system_message)] + 
                         [HumanMessage(content=f"Use this source: {state['context']}")])
    return {"sections": [section.content]}

def write_report(state: ResearchGraphState):
    context = "\n\n".join([f"{s}" for s in state["sections"]])
    system_message = report_writer_instructions.format(topic=state["topic"], context=context)
    report = llm.invoke([SystemMessage(content=system_message)] + [HumanMessage(content="Write report.")])
    return {"content": report.content}

def write_introduction(state: ResearchGraphState):
    context = "\n\n".join([f"{s}" for s in state["sections"]])
    instructions = intro_conclusion_instructions.format(topic=state["topic"], formatted_str_sections=context)
    intro = llm.invoke([instructions] + [HumanMessage(content="Write introduction")])
    return {"introduction": intro.content}

def write_conclusion(state: ResearchGraphState):
    context = "\n\n".join([f"{s}" for s in state["sections"]])
    instructions = intro_conclusion_instructions.format(topic=state["topic"], formatted_str_sections=context)
    conclusion = llm.invoke([instructions] + [HumanMessage(content="Write conclusion")])
    return {"conclusion": conclusion.content}

def finalize_report(state: ResearchGraphState):
    content = state["content"].replace("## Insights", "").strip()
    sources = content.split("\n## Sources\n")[1] if "## Sources" in content else None
    content = content.split("\n## Sources\n")[0] if sources else content
    
    final_report = f"{state['introduction']}\n\n---\n\n{content}\n\n---\n\n{state['conclusion']}"
    if sources: final_report += f"\n\n## Sources\n{sources}"
    return {"final_report": final_report}

### Cell 5: Main Orchestration and Graph Compilation
This cell ties the sub-graph and the main graph together and sets up the Parallel Send logic.

In [7]:
from langgraph.types import Send

def initiate_all_interviews(state: ResearchGraphState):
    if state.get('human_analyst_feedback'):
        return "create_analysts"
    
    topic = state["topic"]
    return [Send("conduct_interview", {
        "analyst": a,
        "messages": [HumanMessage(content=f"So you said you were writing an article on {topic}?")],
        "max_num_turns": 2
    }) for a in state["analysts"]]

# --- Compile Sub-Graph ---
it_builder = StateGraph(InterviewState)
it_builder.add_node("ask_question", generate_question)
it_builder.add_node("search_web", search_web); it_builder.add_node("search_wikipedia", search_wikipedia)
it_builder.add_node("answer_question", generate_answer)
it_builder.add_node("save_interview", save_interview); it_builder.add_node("write_section", write_section)
it_builder.add_edge(START, "ask_question")
it_builder.add_edge("ask_question", "search_web"); it_builder.add_edge("ask_question", "search_wikipedia")
it_builder.add_edge("search_web", "answer_question"); it_builder.add_edge("search_wikipedia", "answer_question")
it_builder.add_conditional_edges("answer_question", route_messages, ['ask_question', 'save_interview'])
it_builder.add_edge("save_interview", "write_section"); it_builder.add_edge("write_section", END)
interview_graph = it_builder.compile()

# --- Compile Main Graph ---
builder = StateGraph(ResearchGraphState)
builder.add_node("create_analysts", create_analysts)
builder.add_node("human_feedback", human_feedback)
builder.add_node("conduct_interview", interview_graph)
builder.add_node("write_report", write_report)
builder.add_node("write_introduction", write_introduction)
builder.add_node("write_conclusion", write_conclusion)
builder.add_node("finalize_report", finalize_report)

builder.add_edge(START, "create_analysts")
builder.add_edge("create_analysts", "human_feedback")
builder.add_conditional_edges("human_feedback", initiate_all_interviews, ["create_analysts", "conduct_interview"])
builder.add_edge("conduct_interview", "write_report")
builder.add_edge("conduct_interview", "write_introduction")
builder.add_edge("conduct_interview", "write_conclusion")
builder.add_edge(["write_conclusion", "write_report", "write_introduction"], "finalize_report")
builder.add_edge("finalize_report", END)

graph = builder.compile(interrupt_before=['human_feedback'], checkpointer=MemorySaver())

### Cell 6: Instructions & Prompts for Research Orchestration
This cell acts as the "Instruction Manual" or "Brain Configuration" for the entire graph.

In [8]:
# --- 1. Analyst Question Generation (used in generate_question) ---
question_instructions = """You are an analyst tasked with interviewing an expert to learn about a specific topic. 

Your goal is boil down to interesting and specific insights related to your topic.

1. Interesting: Insights that people will find surprising or non-obvious.
2. Specific: Insights that avoid generalities and include specific examples from the expert.

Here is your topic of focus and set of goals: {goals}
        
Begin by introducing yourself using a name that fits your persona, and then ask your question.
Continue to ask questions to drill down and refine your understanding of the topic.
        
When you are satisfied with your understanding, complete the interview with: "Thank you so much for your help!"
Remember to stay in character throughout your response, reflecting the persona and goals provided to you."""


# --- 2. Search Query Generation (used in search_web & search_wikipedia) ---
search_instructions = SystemMessage(content="""You will be given a conversation between an analyst and an expert. 

Your goal is to generate a well-structured query for use in retrieval and / or web-search related to the conversation.
        
First, analyze the full conversation. Pay particular attention to the final question posed by the analyst.

Convert this final question into a well-structured web search query""")


# --- 3. Expert Answer Generation (used in generate_answer) ---
answer_instructions = """You are an expert being interviewed by an analyst.

Here is analyst area of focus: {goals}. 
        
You goal is to answer a question posed by the interviewer.
To answer question, use this context:
        
{context}

Guidelines:
1. Use only the information provided in the context. 
2. Do not introduce external information or make assumptions beyond what is explicitly stated in the context.
3. The context contain sources at the topic of each individual document.
4. Include these sources your answer next to any relevant statements (e.g., [1]).
5. List your sources in order at the bottom of your answer. [1] Source 1, [2] Source 2, etc"""


# --- 4. Section Writing (used in write_section) ---
section_writer_instructions = """You are an expert technical writer. 
Your task is to create a short, easily digestible section of a report based on a set of source documents.

1. Create a report structure using markdown formatting (## for Title, ### for headers).
2. Make your title engaging based upon the focus area: {focus}
3. Emphasize what is novel, interesting, or surprising gathered from the interview.
4. Use numbered sources in your report (e.g., [1], [2]).
5. Aim for approximately 400 words maximum."""


# --- 5. Final Report Synthesis (used in write_report) ---
report_writer_instructions = """You are a technical writer creating a report on this overall topic: {topic}
    
Consolidate memos from your analysts into a cohesive narrative.
1. Use markdown formatting. 
2. Include no pre-amble.
3. Start with title header: ## Insights
4. Preserve citations in the memos (e.g., [1]).
5. Create a final, consolidated ## Sources section at the end."""


# --- 6. Intro & Conclusion Writing (used in write_introduction & write_conclusion) ---
intro_conclusion_instructions = """You are a technical writer finishing a report on {topic}

Write a crisp (~100 words) and compelling introduction or conclusion as instructed.
Include no pre-amble.
For Intro: Use # for title and ## Introduction header.
For Conclusion: Use ## Conclusion header.

Sections to reflect on: {formatted_str_sections}"""

## Simulation

### Cell 1: Initial Run (Persona Generation)
This starts the process, generates the analysts, and then stops at the human_feedback node.

In [9]:
# Setup the thread and initial inputs
thread = {"configurable": {"thread_id": "simulation_1"}}
initial_input = {
    "topic": "The benefits of adopting LangGraph as an agent framework",
    "max_analysts": 3
}

print("--- Starting Persona Generation ---")
for event in graph.stream(initial_input, thread, stream_mode="values"):
    analysts = event.get('analysts', [])
    if analysts:
        print(f"\nGenerated {len(analysts)} Analysts:")
        for a in analysts:
            print(f"- {a.name} ({a.role}): {a.affiliation}")

# Verification: The graph should now be 'hibernating'
state = graph.get_state(thread)
print(f"\nNext node to execute: {state.next}")

--- Starting Persona Generation ---

Generated 3 Analysts:
- Dr. Rachel Kim (AI Researcher): Research Institute
- Alex Chen (Software Engineer): Tech Firm
- Dr. Liam Patel (Computer Science Professor): Academic Institution

Next node to execute: ('human_feedback',)


### Cell 2: Providing Feedback (The Loop)
If you aren't happy with the analysts, run this cell to tell the AI what to change. This triggers the conditional_edge to go back to create_analysts.

In [10]:
# Provide feedback to the graph
feedback = "Add a skeptical researcher who is worried about the complexity of state management."

graph.update_state(
    thread, 
    {"human_analyst_feedback": feedback}, 
    as_node="human_feedback"
)

print("--- Feedback Sent. Regenerating Analysts... ---")
# Resume execution
for event in graph.stream(None, thread, stream_mode="values"):
    analysts = event.get('analysts', [])
    if analysts:
        for a in analysts:
            print(f"Updated Name: {a.name} | Role: {a.role}")

--- Feedback Sent. Regenerating Analysts... ---
Updated Name: Dr. Rachel Kim | Role: AI Researcher
Updated Name: Alex Chen | Role: Software Engineer
Updated Name: Dr. Liam Patel | Role: Computer Science Professor
Updated Name: Dr. Rachel Kim | Role: AI Research Analyst
Updated Name: Dr. Eric Thompson | Role: AI Research Analyst
Updated Name: Dr. Maria Rodriguez | Role: AI Research Analyst
Updated Name: Dr. David Lee | Role: AI Research Analyst
Updated Name: Dr. Sophia Patel | Role: AI Research Analyst


### Cell 3: Approving and Executing Parallel Interviews
Once you are happy, you must clear the feedback to satisfy the initiate_all_interviews logic. This will kick off the parallel Send API calls to the interview sub-graphs.

In [ ]:
# Signal approval by setting feedback to None/Empty
graph.update_state(
    thread, 
    {"human_analyst_feedback": None}, 
    as_node="human_feedback"
)

print("--- Approval Received. Starting Parallel Interviews & Report Writing... ---")
print("(This may take a minute as multiple analysts are researching simultaneously)")

# We use stream_mode="updates" here so you can see the nodes finishing in real-time
for event in graph.stream(None, thread, stream_mode="updates"):
    for node_name, data in event.items():
        print(f"Finished Node: {node_name}")
        
print("\n--- Research Complete! ---")

--- Approval Received. Starting Parallel Interviews & Report Writing... ---
(This may take a minute as multiple analysts are researching simultaneously)


### Cell 4: Viewing the Final Report
After the parallel work is done, the "Reduce" nodes (write_report, write_introduction, etc.) will have finished. Run this to see the result.

In [ ]:
from IPython.display import Markdown

# Get the final state from the thread
final_state = graph.get_state(thread)
report_content = final_state.values.get('final_report')

if report_content:
    display(Markdown(report_content))
else:
    print("Report not found. Ensure the graph finished the finalize_report node.")

We can look at the trace:

https://smith.langchain.com/public/2933a7bb-bcef-4d2d-9b85-cc735b22ca0c/r